<a href="https://colab.research.google.com/github/BrotherKim/KAIST_CODE/blob/master/SEP531/term/5_generate_testset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#변수 정의

In [12]:
REGION_LIST = ['ks', 'jr', 'kw', 'cc'] # jj는 특이케이스라 따로 처리

ROOT_DIR = '/content/drive/MyDrive/KAIST/SEP531/kor_%s_labeled'
TRAIN_DIR = '%s/train' % ROOT_DIR
VALID_DIR = '%s/valid' % ROOT_DIR

NUM_TRAIN = 171
NUM_VALID = 58

SAVEPOINT_PATH = '/content/drive/MyDrive/KAIST/SEP531/KoBERT_emotion_finetuned_data_std.pt'

#실행 환경#

- Python >= 3.6
- PyTorch >= 1.70
- Transformers = 3.0.2
- Colab
- batch size = 64 (convertable)
- epochs = 10 (convertable)

In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

     |████████████████████████████████| 46.9 MB 2.0 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595737 sha256=87e9ea93d4f252f2243363ff3a769c6927f42f9a9e2057d53747610b9305049f
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.2 MB/s 
     |████████████████████████████████| 769 kB 5.3 MB/s 
     |████████████████████████████████| 3.0 MB 53.4 MB/s 
     |████████████████████████████████| 895 kB 61.1 MB/s 


#KoBERT 다운로드#

In [3]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-kdwmvqtu
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-kdwmvqtu
     |████████████████████████████████| 4.9 MB 5.0 MB/s 
     |████████████████████████████████| 3.3 MB 48.4 MB/s 
     |████████████████████████████████| 3.3 MB 53.1 MB/s 
     |████████████████████████████████| 596 kB 72.2 MB/s 
     |████████████████████████████████| 61 kB 650 kB/s 
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=13136 sha256=2c9c1bda297695daf2f78920be00a3ec89104fc83544a8e13614b1ac0e586e43
  Stored in directory: /tmp/pip-ephem-wheel-cache-3e3gzger/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Attempting uninstall: tokenizers
    Found existi

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
#GPU 사용
device = torch.device("cuda:0")

In [7]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

.cache/kobert_v1.zip[██████████████████████████████████████████████████]
.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


#데이터셋 전처리#

In [8]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
# 터미널 커맨드를 문자열로 반환해주는 함수 작성

import subprocess

def GetShellCmdStdOut(command):
  cmd = ['sh', '-c', command]
  fd_popen = subprocess.Popen(cmd, stdout=subprocess.PIPE).stdout 
  data = fd_popen.read().strip() 
  fd_popen.close()

  retval = data.decode('utf-8') 
  return retval

In [10]:
import pandas as pd

def GenDataset(DIR, json_list):
  retval = []

  for fn in json_list:
    #print('%s\n' % (fn))
    fd = pd.read_csv('%s/%s' % (DIR, fn), sep='\t', header=None)
    # [0]데이터가 표준어, [1]데이터가 사투리, [2]가 감정
    for q, label in zip(fd[0], fd[2])  :
      #print('%s, %s\n' % (q, label))
      data = []
      data.append(q)
      data.append(str(label))
      retval.append(data)

  return retval

In [13]:
dataset_train = []

for region in REGION_LIST:
  train_dir = TRAIN_DIR % (region)
  print(train_dir)
  %pushd $train_dir
  tl = GetShellCmdStdOut('find | grep answer').split('\n')
  train_json_list = [x.replace('./', '') for x in tl]
  dataset_add = GenDataset(train_dir, train_json_list)
  #dataset_train = dataset_train + dataset_add[:11250]
  dataset_train = dataset_train + dataset_add[45000:]
  %popd

dataset_valid = []

for region in REGION_LIST:
  valid_dir = VALID_DIR % (region)
  print(valid_dir)
  %pushd $valid_dir
  vl = GetShellCmdStdOut('find | grep answer').split('\n')
  valid_json_list = [x.replace('./', '') for x in vl]
  dataset_add = GenDataset(valid_dir, valid_json_list)
  #dataset_valid = dataset_valid + dataset_add[:3750]
  dataset_valid = dataset_valid + dataset_add[15000:]
  %popd

/content/drive/MyDrive/KAIST/SEP531/kor_ks_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_ks_labeled/train
/content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
popd -> /content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_jr_labeled/train
/content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
popd -> /content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_kw_labeled/train
/content/drive/My Drive/KAIST/SEP531/kor_kw_labeled/train
/content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
popd -> /content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_cc_labeled/train
/content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
popd -> /content/drive/My Drive/KAIST/SEP531/kor_ks_labeled/train
/content/drive/MyDrive/KAIST/SEP531/kor_ks_labe

In [14]:
print(len(dataset_train))
print(len(dataset_valid))

78559
55144


In [15]:
dataset_train[:10]

[['많이 지치긴 지치드라.', '3'],
 ['요즘도 요즘 뭐~', '4'],
 ['코로나 터지고는 헬스장 운영 안 해 가지고 한 별로 안 가고 못 가고 이랬는데', '3'],
 ['그래도 또 하면 조금 약간 스트레스 풀리더라고 그래서', '4'],
 ['-꾸준- 꾸준히 할까 생각 중이기도 하고', '4'],
 ['근데 그거 하면은 다리 알 생기고 그런 거 아니지?', '0'],
 ['아~ 그런 거 아니야 그냥 확실히 다리하고 다릿 살은 허벅지살 특히 많이 빠져.', '4'],
 ['그런 거 하면 좋드라 그러니까', '4'],
 ['하면 확실히 여자분 들은 이~ 다리 지방 많은 그~ 다리 이쪽이 많이 빠지드라 (()) 허벅지', '3'],
 ['힘이 많이 들어가서 많이 다리는 조금 빠지는 거 같', '3']]

In [17]:
with open('std_test.csv', 'w') as f:
  for d, l in dataset_train:
    f.write('%s, %s' % (d, l))
  for d, l in dataset_valid:
    f.write('%s, %s' % (d, l))

In [18]:
!cp std_test.csv /content/drive/MyDrive/KAIST/SEP531